In [1]:
import pandas as pd
import seaborn as sns
import random

In [2]:
import io

In [18]:
movie = pd.read_csv("/content/movies.csv")

In [19]:
rating = pd.read_csv("/content/ratings.csv")

In [20]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [21]:
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [22]:
df_ = movie.merge(rating, how="left", on="movieId")
df = df_.copy()
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [23]:
def create_user_movie_df():
    rating_counts = pd.DataFrame(df["title"].value_counts())
    rating_counts.describe([0.05, 0.50, 0.75, 0.85, 0.90, 0.95, 0.99]).T
    limit = rating_counts.quantile([0.90]).T
    limit_90 = limit[0.9][0]
    rare_movies = rating_counts[rating_counts["title"] <= limit_90].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index="userId", columns="title", values="rating")
    return user_movie_df

In [24]:
user_movie_df = create_user_movie_df()
user_movie_df.head() 

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",16 Blocks (2006),...,Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zelig (1983),Zodiac (2007),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
def random_user_choose(user_movie_df, random_state):
    random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=random_state).values)
    return random_user

In [26]:
random_user = random_user_choose(user_movie_df, random_state=33)
random_user

286

In [27]:
def movie_id_actual():
    movie_id_act = rating[(rating["userId"] == random_user) & (rating["rating"] >= 4.0)]. \
        sort_values(by="timestamp", ascending=False)["movieId"][0:1].tolist()
    return movie_id_act[0]

In [28]:
movie_id = movie_id_actual()
movie_id

500

In [29]:
def movie_id_name(dataframe, movie_ID):
    movie_name = dataframe[dataframe["movieId"] == movie_ID][["title"]].values[0].tolist()
    return movie_name[0]

In [30]:
movie_name = movie_id_name(df, movie_id)
movie_name

'Mrs. Doubtfire (1993)'

In [31]:
def item_based_recommender(movie_name, user_movie_df, random_user):
    movie_name = user_movie_df[movie_name]
    recommend_list = user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(20)
    recommend_list = recommend_list.reset_index()
    recommend_list = recommend_list["title"].tolist()
    recommend_list

    random_user_df = user_movie_df[user_movie_df.index == random_user]
    random_user_movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
    random_user_movies_watched_df = user_movie_df[random_user_movies_watched]
    random_user_movies_watched_df = random_user_movies_watched_df.columns.tolist()

    movie_to_recommend = [i for i in random_user_movies_watched_df if i not in recommend_list]
    movie_to_recommend = random.sample(movie_to_recommend, 5)
    return movie_to_recommend

In [32]:
item_based_recommender(movie_name, user_movie_df, random_user)

/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


['Net, The (1995)',
 'Santa Clause, The (1994)',
 'Congo (1995)',
 'GoldenEye (1995)',
 'Nell (1994)']

In [33]:
def movie_name(dataframe, movieId_list):
    for i in movieId_list:
        movie_name = dataframe[dataframe["movieId"] == i][["title"]].values[0].tolist()
        print(movie_name)

In [34]:
def user_based_recommender(user_movie_df, random_user):
    random_user_df = user_movie_df[user_movie_df.index == random_user]
    random_user_movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
    movies_watched_df = user_movie_df[random_user_movies_watched]

    user_movie_count = movies_watched_df.T.notnull().sum()
    user_movie_count = user_movie_count.reset_index()
    user_movie_count.columns = ["userId", "movie_count"]

    user_movie_count.describe([0.05, 0.50, 0.75, 0.85, 0.90, 0.95, 0.99]).T
    limit_movie = user_movie_count.quantile([0.99]).T
    limit_movie_99 = limit_movie[0.99][1]

    users_same_movies = user_movie_count[user_movie_count["movie_count"] >= limit_movie_99]["userId"]
    final_df = movies_watched_df[movies_watched_df.index.isin(users_same_movies)]
    
    final_df.T.corr()
    corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
    corr_df = pd.DataFrame(corr_df, columns=["corr"])
    corr_df.index.names = ["userId_1", "userId_2"]
    corr_df = corr_df.reset_index()

    same_users = corr_df[(corr_df["userId_1"] == random_user)]
    same_users.describe([0.05, 0.50, 0.75, 0.85, 0.90, 0.95, 0.99]).T
    limit_same_user = same_users.quantile([0.95]).T
    limit_same_users_95 = limit_same_user[0.95][2]

    top_users = corr_df[(corr_df["userId_1"] == random_user) & (corr_df["corr"] >= limit_same_users_95)][["userId_2", "corr"]].reset_index(drop=True)
    top_users = top_users.sort_values(by="corr", ascending=False)
    top_users.rename(columns={"userId_2": "userId"}, inplace=True)
    top_users_rating = top_users.merge(rating[["userId", "movieId", "rating"]], how="inner")
    top_users_rating = top_users_rating[top_users_rating["userId"] != random_user]

    top_users_rating["weighted_rating"] = top_users_rating["corr"] * top_users_rating["rating"]
    recommendation_df = top_users_rating.groupby("movieId")["weighted_rating"].mean()
    recommendation_df = recommendation_df.reset_index()
    recommendation_df_5 = recommendation_df.sort_values(by="weighted_rating", ascending=False).head(5)
    recommendation_df_5_list = recommendation_df_5["movieId"].tolist()

    return movie_name(df, recommendation_df_5_list)

In [35]:
user_based_recommender(user_movie_df, random_user)

["William Shakespeare's Romeo + Juliet (1996)"]
['Jerry Maguire (1996)']
['Die Hard (1988)']
['Scary Movie (2000)']
['Young Guns (1988)']
